In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import train_test_split
# from sklearn.decomposition import PCA
# from sklearn.neural_network import MLPClassifier
# from sklearn.linear_model import Perceptron
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
# from  sklearn.preprocessing  import StandardScaler

In [2]:
df = pd.read_csv('../data/final_data/entrenamiento_postulaciones_vistas4.csv', encoding='utf-8')
df.head()

,idaviso,idpostulante,timestamp,valor_educacion,sexo,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo
0,1111556097,365446,7.038007e+06,17,1,0,0,0,0,0,0,0,0,1
1,1112370711,365446,7.476885e+06,17,1,6996,0,0,0,0,0,45,14,1
2,1112326655,365446,9.809277e+06,17,1,11827,0,0,0,0,0,4,33,1
3,1112350182,365446,8.856385e+06,17,1,2141,0,0,0,0,0,31,38,1
4,1112383837,365446,7.563903e+06,17,1,6942,0,0,0,0,0,75,46,1


In [3]:
n_features = 13
dfix = df.iloc[:, 0:n_features];
X = np.array(dfix)
y = np.array(df['se_postulo']) 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=7506)

In [5]:
clf1 = tree.DecisionTreeClassifier()
clf2 = GradientBoostingClassifier()
clf3 = VotingClassifier(estimators=[('tree', clf1), ('gb', clf2)])
clf4 = GaussianNB()
clf = VotingClassifier(estimators=[('tree-gb', clf3), ('gnb', clf4)])
clf.fit(X_train, y_train)

VotingClassifier(estimators=[('mlp-gb', VotingClassifier(estimators=[('mlp', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_spli...  flatten_transform=None, n_jobs=1, voting='hard', weights=None)), ('gnb', GaussianNB(priors=None))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [6]:
pred = clf.predict(X_test)

/home/prp-admin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/prp-admin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
print (accuracy_score(y_test, pred))

0.8220594003483063


In [8]:
pred.shape

(956055,)

In [9]:
X_full_data = np.array(dfix)
pred_full_data = clf.predict(X_full_data)
df['sepostulo'] = pred_full_data
df.head()

/home/prp-admin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/prp-admin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,idaviso,idpostulante,timestamp,valor_educacion,sexo,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo,sepostulo
0,1111556097,365446,7.038007e+06,17,1,0,0,0,0,0,0,0,0,1,1
1,1112370711,365446,7.476885e+06,17,1,6996,0,0,0,0,0,45,14,1,1
2,1112326655,365446,9.809277e+06,17,1,11827,0,0,0,0,0,4,33,1,1
3,1112350182,365446,8.856385e+06,17,1,2141,0,0,0,0,0,31,38,1,1
4,1112383837,365446,7.563903e+06,17,1,6942,0,0,0,0,0,75,46,1,1


## Subida 

In [10]:
top100 = pd.read_csv("../test_final_100k.csv", encoding='utf-8')
id_postulantes = pd.read_csv('../data/final_data/entrenamiento_id_postulantes.csv', encoding='utf-8')
id_postulantes.drop('Unnamed: 0', axis=1, inplace=True)
top100_full = pd.merge(top100, id_postulantes, how='left', on='idpostulante')
top100_full.drop('idpostulante', inplace=True, axis=1)
top100_full.rename({'idpostulante2':'idpostulante'}, inplace=True, axis=1)
top100_full = pd.merge(top100_full, df, how='left', on=['idaviso','idpostulante'])
top100_full = top100_full[['id', 'se_postulo', 'sepostulo']]
top100_full.fillna(0, inplace=True)
top100_full['se_postulo'] = top100_full['se_postulo'].astype('int')
top100_full['sepostulo'] = top100_full['sepostulo'].astype('int')
top100_full.loc[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1), 'sepostulo'] = 1
top100_full[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1)]['sepostulo']
top100_full.drop('se_postulo', axis=1, inplace=True)
top100_full[top100_full['sepostulo'] == 1].shape

(33546, 2)

In [11]:
top100_full.to_csv('../data/entrega/top100k_ensemble_tree_gb_gnb.csv', encoding='utf-8', index=False)